In [ ]:
# Copyright 2022 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

删除Vertex AI TensorBoard中过时的实验

<table align="left">
  <td style="text-align: center">
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/official/experiments/delete_outdated_tensorboard_experiments.ipynb">
      <img width="32px" src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Google Colaboratory logo"><br> 在Colab中打开
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/colab/import/https:%2F%2Fraw.githubusercontent.com%2FGoogleCloudPlatform%2Fvertex-ai-samples%2Fmain%2Fnotebooks%2Fofficial%2Fexperiments%2Fdelete_outdated_tensorboard_experiments.ipynb">
      <img width="32px" src="https://cloud.google.com/ml-engine/images/colab-enterprise-logo-32px.png" alt="Google Cloud Colab Enterprise logo"><br> 在Colab Enterprise中打开
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/vertex-ai-samples/main/notebooks/official/experiments/delete_outdated_tensorboard_experiments.ipynb">
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo"><br> 在Vertex AI Workbench中打开
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/official/experiments/delete_outdated_tensorboard_experiments.ipynb">
      <img width="32px" src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo"><br> 在GitHub上查看
    </a>
  </td>
</table>

注意：此笔记本已在以下环境中进行了测试：

- Python版本=3.9

## 综述

Vertex AI 有一个新的 Vertex AI TensorBoard 计费模型。费用基于每月存储成本为 10 美元/GB。因此，删除旧的 Vertex AI TensorBoard 实验是明智的，以避免不必要的存储成本。

了解更多关于 [Vertex AI TensorBoard](https://cloud.google.com/vertex-ai/docs/experiments/tensorboard-introduction)。

### 目标

在本教程中，您将学习如何删除过时的TensorBoard实验，以避免不必要的存储成本。

本教程使用以下Google Cloud ML服务和资源：

- Vertex AI TensorBoard

执行的步骤包括：

- 如何使用预定义的键值标签对 `<label_key, label_value>` 删除TB实验

- 如何删除在 `create_time` 之前创建的TB实验

- 如何删除在 `update_time` 之前创建的TB实验

### 数据集

没有使用数据集。

费用

本教程使用 Google Cloud 的计费组件：

* Vertex AI
* Cloud Storage

了解 [Vertex AI 定价](https://cloud.google.com/vertex-ai/pricing)，以及 [Cloud Storage 定价](https://cloud.google.com/storage/pricing)，
并使用 [定价计算器](https://cloud.google.com/products/calculator/)
根据您的预期使用量生成成本估算。

开始吧

### 为Python安装Vertex AI SDK和其他必需的软件包。

In [ ]:
# Install the packages
import os

if not os.getenv("IS_TESTING"):
    USER = "--user"
else:
    USER = ""
! pip3 install {USER} --upgrade google-cloud-aiplatform

### 仅适用于Colab：取消注释### 重新启动运行时（仅适用于Colab）

要使用新安装的软件包，您必须重新启动Google Colab上的运行时。请在下一个单元格中重新启动内核。

In [ ]:
import sys

if "google.colab" in sys.modules:

    import IPython

    app = IPython.Application.instance()
    app.kernel.do_shutdown(True)

<div class =“ alert alert-block alert-warning”>
<b> ⚠️ 内核将重新启动。请等待完成后再继续下一步。⚠️ </b>
</ div>

### 在Google Colab上验证您的笔记本环境

请在Google Colab上验证您的环境。

In [ ]:
import sys

if "google.colab" in sys.modules:

    from google.colab import auth

    auth.authenticate_user()

设置 Google 云项目信息并初始化用于 Python 的 Vertex AI SDK

要开始使用 Vertex AI，您必须拥有现有的 Google 云项目并[启用 Vertex AI API](https://console.cloud.google.com/flows/enableapi?apiid=aiplatform.googleapis.com)。了解有关[设置项目和开发环境](https://cloud.google.com/vertex-ai/docs/start/cloud-environment)的更多信息。

In [ ]:
PROJECT_ID = "[your-project-id]"  # @param {type:"string"}
LOCATION = "us-central1"  # @param {type:"string"}


from google.cloud import aiplatform

aiplatform.init(project=PROJECT_ID, location=LOCATION)

### 定义常量

在本教程中定义您要使用的变量。具体来说，您设置

- `CREATE_TIME_CUT` : 删除在 CREATE_TIME_CUT 之前创建的 TensorBoard 实验。例如，`2022-12-31`。

- `UPDATE_TIME_CUT` : 删除在 UPDATE_TIME_CUT 之前创建的 TensorBoard 实验。例如，`2022-12-31`。

- `DETAILED_LOG` : 一个布尔变量，用于查看 TensorBoard 删除进度。如果为 True，则按实验显示进度。否则，它每100个实验报告进度。

In [ ]:
CREATE_TIME_CUT = ""  # @param {type:"string"}

UPDATE_TIME_CUT = ""  # @param {type:"string"}

DETAILED_LOG = True  # @param {type: "boolean"}

定义帮助者

In [ ]:
def clean_up_by_label(tensorboard_instance, label_key, label_value):
    """Delete the TB Experiment with the key-value label pair <label_key, label_value>"""
    # List tensorboard experiments
    tensorboard_experiments = aiplatform.TensorboardExperiment.list(
        tensorboard_name=tensorboard_instance.resource_name
    )
    # Get the number of tensorboard experiments
    num_tensorboard_experiments = len(tensorboard_experiments)
    # For each experiment
    for i in range(num_tensorboard_experiments):
        tensorboard_experiment = tensorboard_experiments[i]
        if DETAILED_LOG or (i % 100 == 0):
            print(
                f">>>checking TB experiment [{i + 1}/{num_tensorboard_experiments}]: {tensorboard_experiment.resource_name}"
            )
        # Get experiment labels
        labels = tensorboard_experiment.labels
        # Filter by label
        if label_key in labels and labels[label_key] == label_value:
            # Delete experiment
            tensorboard_experiment.delete()


def clean_up_by_create_time(tensorboard_instance, create_time_cut):
    """Delete the TB Experiment with the `create_time`"""
    # List tensorboard experiments
    tensorboard_experiments = aiplatform.TensorboardExperiment.list(
        tensorboard_name=tensorboard_instance.resource_name, order_by="create_time"
    )
    # Get the number of tensorboard experiments
    num_tensorboard_experiments = len(tensorboard_experiments)
    # For each experiment
    for i in range(num_tensorboard_experiments):
        tensorboard_experiment = tensorboard_experiments[i]
        if DETAILED_LOG or (i % 100 == 0):
            print(
                f">>> checking TB experiment [{i + 1}/{num_tensorboard_experiments}]: {tensorboard_experiment.resource_name}"
            )
        # Filter by create_time
        if str(tensorboard_experiment.create_time) < create_time_cut:
            # Delete experiment
            tensorboard_experiment.delete()
        else:
            break


def clean_up_by_update_time(tensorboard_instance, update_time_cut):
    """Delete the TB Experiment with with the `update_time`"""
    # List tensorboard experiments
    tensorboard_experiments = aiplatform.TensorboardExperiment.list(
        tensorboard_name=tensorboard_instance.resource_name, order_by="update_time"
    )
    # Get the number of tensorboard experiments
    num_tensorboard_experiments = len(tensorboard_experiments)
    # For each experiment
    for i in range(num_tensorboard_experiments):
        tensorboard_experiment = tensorboard_experiments[i]
        if DETAILED_LOG or (i % 100 == 0):
            print(
                f">>> checking TB experiment [{i + 1}/{num_tensorboard_experiments}]: {tensorboard_experiment.resource_name}"
            )
        # Filter by update_time
        if str(tensorboard_experiment.update_time) < update_time_cut:
            tensorboard_experiment.delete()
        else:
            break

删除过时的Vertex AI TensorBoard实验

### 设置删除过期的TensorBoard实验

初始化一个标志变量来开始删除过期的TensorBoard实验，以及一个标志变量来选择删除方法。

In [ ]:
delete_tb_experiments = False  # @param {type: "boolean"}

delete_method = ""  # @param ["by_label", "by_create_time", "by_update_time"]

### 删除一个TensorBoard实例

要删除一个TensorBoard实例，您需要`TENSORBOARD_INSTANCE` ID，它唯一标识您运行实验的TensorBoard实例。

要获取TensorBoard实例ID，您可以选择：

- 转到云控制台UI，Vertex AI > 实验 > TensorBoard实例，或
- 使用下面的列表命令列出您项目和地区的所有TensorBoard实例。

In [ ]:
tensorboard_instances = aiplatform.Tensorboard.list(
    project=PROJECT_ID, location=LOCATION
)
print(tensorboard_instances)

设置要删除实验的tensorboard实例ID。

In [ ]:
if delete_tb_experiments:

    TENSORBOARD_INSTANCE_ID = ""  # @param {type:"string"}

    TENSORBOARD_INSTANCE = aiplatform.Tensorboard(
        project=PROJECT_ID, location=LOCATION, tensorboard_name=TENSORBOARD_INSTANCE_ID
    )
    print(TENSORBOARD_INSTANCE)

使用键-值标签对删除TensorBoard实验

您可以使用预定义的`label_key`和`label_value`删除TensorBoard实验。
例如，您可以指定`delete`标签键和`true`标签值来表示您想要删除的所有TensorBoard实验。
TensorBoard实验标签可以在创建时设置，也可以使用API更新方法设置。

In [ ]:
LABEL_KEY = "delete"  # @param {type:"string"}
LABEL_VALUE = "true"  # @param {type:"string"}

In [ ]:
if delete_tb_experiments and delete_method == "by_label":
    clean_up_by_label(TENSORBOARD_INSTANCE, LABEL_KEY, LABEL_VALUE)

使用`create_time`字段删除一个TensorBoard实验

In [ ]:
if delete_tb_experiments and delete_method == "by_create_time":
    clean_up_by_create_time(TENSORBOARD_INSTANCE, CREATE_TIME_CUT)

使用预定义的`update_time`字段删除一个TensorBoard实验

In [ ]:
if delete_tb_experiments and delete_method == "by_update_time":
    clean_up_by_update_time(TENSORBOARD_INSTANCE, UPDATE_TIME_CUT)